# Model Disagreement Analysis

This notebook analyzes the output CSVs from the inference pipeline.

In [ ]:
import pandas as pd
import ast
import matplotlib.pyplot as plt

csv_path = "../outputs/cifar10.csv" # Example path

In [ ]:
try:
    df = pd.read_csv(csv_path)
    print(f"Loaded {len(df)} samples")
except FileNotFoundError:
    print("File not found. Run inference first.")

In [ ]:
if 'df' in locals():
    # Accuracy
    acc = (df['prediction'] == df['ground_truth']).mean()
    print(f"Global Accuracy: {acc:.4f}")
    
    # Confidence hist
    df['confidence'].hist(bins=20)
    plt.title("Confidence Distribution")
    plt.show()